# Using waterbodies for basin and national scale assessment

* **Products used:** 
[ls8_sr](https://explorer.digitalearth.africa/ls8_sr), 
[wofs_ls_summary_annual](https://explorer.digitalearth.africa/wofs_ls_summary_annual),
* **Special requirements:** An _optional_ description of any special requirements
* **Prerequisites:** An _optional_ list of any notebooks that should be run or content that should be understood prior to launching this notebook


## Background
An *optional* overview of the scientific, economic or environmental management issue or challenge being addressed by Digital Earth Africa. 
For `Beginners_Guide` or `Frequently_Used_Code` notebooks, this may include information about why the particular technique or approach is useful or required. 
If you need to cite a scientific paper or link to a website, use a persistent DOI link if possible and link in-text (e.g. [Dhu et al. 2017](https://doi.org/10.1080/20964471.2017.1402490)).

## Description
A _compulsory_ description of the notebook, including a brief overview of how Digital Earth Africa helps to address the problem set out above.
It can be good to include a run-down of the tools/methods that will be demonstrated in the notebook:

1. First we do this
2. Then we do this
3. Finally we do this

***

## Getting started

Provide any particular instructions that the user might need, e.g. To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Import Python packages that are used for the analysis.

Use standard import commands; some are shown below. 
Begin with any `iPython` magic commands, followed by standard Python packages, then any additional functionality you need from the `Tools` package.

In [1]:
import matplotlib.pyplot as plt
import datacube
import dask
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.animation as animation

from deafrica_tools.plotting import display_map
from IPython.display import HTML
from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.dask import create_local_dask_cluster

from deafrica_tools.waterbodies import get_waterbodies, get_time_series

### Connect to the datacube

Connect to the datacube so we can access DE Africa data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [2]:
dc = datacube.Datacube(app="Waterbody-basin")

In [3]:
create_local_dask_cluster()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/mickwelli@bigpond.com/proxy/8787/status,
Dashboard: /user/mickwelli@bigpond.com/proxy/8787/status,Workers: 1
Total threads: 2,Total memory: 11.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40603,Workers: 1
Dashboard: /user/mickwelli@bigpond.com/proxy/8787/status,Total threads: 2
Started: Just now,Total memory: 11.21 GiB
Comm: tcp://127.0.0.1:35173,Total threads: 2
Dashboard: /user/mickwelli@bigpond.com/proxy/34543/status,Memory: 11.21 GiB
Nanny: tcp://127.0.0.1:34199,


### Analysis parameters

An *optional* section to inform the user of any parameters they'll need to configure to run the notebook:

* `param_name_1`: Simple description (e.g. `example_value`). Advice about appropriate values to choose for this parameter.
* `param_name_2`: Simple description (e.g. `example_value`). Advice about appropriate values to choose for this parameter.


In [4]:
basins = gpd.read_file("../Supplementary_data/Waterbodies/HydroBasins_L3/hybas_lake_af_lev03_v1c.geojson")

In [5]:
zambezi = basins[basins["SORT"] == 6]

In [6]:
bbox = zambezi.bounds
bbox = (bbox.minx.values[0], bbox.miny.values[0], bbox.maxx.values[0], bbox.maxy.values[0])

In [7]:
zambezi_waterbodies = get_waterbodies(bbox, crs="EPSG:4326").clip(zambezi)

In [ ]:
#zambezi_waterbodies.explore()

In [8]:
zambezi_waterbodies.uid.iloc[0]

'ks8wp41gvn'

In [9]:
zambezi_ts = get_time_series(zambezi_waterbodies.uid.iloc[0])
zambezi_ts['uid']=zambezi_waterbodies.uid.iloc[0]

In [10]:
zambezi_ts

,area_wet_m2,percent_wet,area_dry_m2,percent_dry,area_invalid_m2,percent_invalid,area_observed_m2,percent_observed,percent_wet_rolling_median,uid
date,,,,,,,,,,
1984-06-02,0.0,0.0,17100.0,100.0,0.0,0.0,17100.0,100.0,NaN,ks8wp41gvn
1984-09-06,0.0,0.0,17100.0,100.0,0.0,0.0,17100.0,100.0,NaN,ks8wp41gvn
1984-09-22,0.0,0.0,17100.0,100.0,0.0,0.0,17100.0,100.0,0.0,ks8wp41gvn
1984-10-08,0.0,0.0,17100.0,100.0,0.0,0.0,17100.0,100.0,0.0,ks8wp41gvn
1986-12-17,0.0,0.0,17100.0,100.0,0.0,0.0,17100.0,100.0,0.0,ks8wp41gvn
...,...,...,...,...,...,...,...,...,...,...
2024-10-30,0.0,0.0,17100.0,100.0,0.0,0.0,17100.0,100.0,0.0,ks8wp41gvn
2024-11-15,0.0,0.0,17100.0,100.0,0.0,0.0,17100.0,100.0,0.0,ks8wp41gvn
2024-12-01,0.0,0.0,17100.0,100.0,0.0,0.0,17100.0,100.0,0.0,ks8wp41gvn


In [11]:
df = []
for i in range(len(zambezi_waterbodies.uid)):
    df.append(dask.delayed(get_time_series)(
        zambezi_waterbodies.uid.iloc[i]).assign(
        uid=zambezi_waterbodies.uid.iloc[i]))

In [ ]:
dask.compute(*df)

/opt/venv/lib/python3.12/site-packages/distributed/client.py:3361: UserWarning: Sending large graph of size 12.95 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [ ]:
pd.concat(df.compute)

## Heading 1
Use headings to break up key steps/stages of the notebook.

Use markdown text for detailed, descriptive text explaining what the code below does and why it is needed.

> **Note:** Use this markdown format (sparingly) to draw particular attention to an important point or caveat

In [ ]:
# Use code comments for low-level documentation of code
a = 1

### Subheading 1
Use subheadings to break up steps within a single section.

In [ ]:
# Use code comments for low-level documentation of code
b = 2

## Heading 2
Use markdown text for detailed, descriptive text explaining what the code below does and why it is needed.

In [ ]:
# Use code comments for low-level documentation of code
c = 3

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

**Last Tested:**

In [ ]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')